In [31]:
class Station:
    def __init__(self, name):
        self.name = name
        self.delay = 0 # total delay at the station at the current time step
        self.incoming_trains = []  # List of trains coming to this station at the current time step
        # no outgoing connections are stored, since we focus on only the incoming connections

    def add_incoming_train(self, train):
        self.incoming_trains.append(train)
        delay = train.get_delay()
        self.delay += delay
        print(f"Incoming train {train.id} with delay {delay} added to station {self.name}., Station delay updated to {self.delay}.")

    def receive_train(self, train):
        """When a train arrives at the station, update delay and remove train from system."""
        self.delay -= train.get_delay()
        self.incoming_trains.remove(train)
        print(f"Train {train.id} arrived at {self.name} with delay {train.delay}. Station delay updated to {self.delay}.")
    


In [32]:

class Train:
    def __init__(self, id, stops, delay=0):
        print(f"Train {id} created in train with delay {delay}.")
        self.id = id
        self.stops = stops # List of stops the train will m ake, and the time it will take to travel between them, [(station, travel_time, buffer_time), ...] station 0 is the starting station and the travel time is 0, since the train starts there
        self.next_stop_index = 0 # Index of the next stop
        self.from_station = None#stops[0][0] #object of the outgoing station
        self.to_station = None #stops[0][0] #objject of the station the train is going to
        self.delay = delay  # Delay carried by the train
        self.current_station = stops[0][0]
        self.running = False
        self.travel_time_remaining = None  # Set when the train starts traveling, time until the train should arrive at the station

    def start_journey(self):
        """Train starts its journey from the start station."""
        self.start_from_current_station()
        self.running = True
        print(f"Train {self.id} has started its journey from {self.from_station.name} to {self.to_station.name}.")

    def start_from_current_station(self):
        """Move the train to the next stop."""
        #check that the train has not reached the final destination
        if self.next_stop_index > len(self.stops):
            print(f"Train {self.id} has reached its final destination.")
            return
        # Set the from station as the current station
        self.from_station = self.stops[self.next_stop_index][0]
        # Set the next station as the station after the current station
        next_station = self.stops[self.next_stop_index + 1]
        #set to station object to the next station and the travel time remaining to the time it takes to get there
        self.to_station = next_station[0]
        self.travel_time_remaining = next_station[1]

        # add the train to the incoming trains of the next station
        next_station[0].add_incoming_train(self)
        # Update the next stop index
        self.next_stop_index += 1
        self.running = True
    
    def update_travel(self):
        """Simulate train moving closer to destination."""
        if self.travel_time_remaining > 0:
            self.travel_time_remaining -= 1  # Simulate time passage
            if self.travel_time_remaining == 0:
                if self.running:
                    self.arrive_at_destination()
                    self.travel_time_remaining = self.stops[self.next_stop_index][2] #set the travel time remaining to the buffer time
                    self.running = False
                else: # Train should leave the station
                    self.start_from_current_station()

    def arrive_at_destination(self):
        """The train arrives at the destination station."""
        self.to_station.receive_train(self)  # Pass the delay to the destination station
        print(f"Train {self.id} has arrived at {self.to_station.name}.")
        # should we do the updating of station index here instead?
        

    def get_delay(self):
        return self.delay

# Questions
# 1. how do we look at trains that pass the station but do not stop? It says something about it in the model description, do we need to add their delays? probably not, right?
# 2. how do we know that a train is delayed from the data? is it the difference between the scheduled and actual arrival time? or should we calculate on the road? or should we just use the delay to calculate a bunch of stuff and use that for the model? So the delay times are only needed for the model, not for the actual train object. which means that the train object is not even needed?
# 3. follow up question to 2. what are we even modelling? What should be the input and output of the model?
    # like shuold it be real time like what are the delays at the moment based on the CURRENT delays? or should it be like what are the delays at the moment based on the delays that were inputted in the model?

In [33]:

class Network:
    def __init__(self):
        self.stations = {}
        self.trains = []
    
    def add_station(self, name):
        self.stations[name] = Station(name)
    
    def add_train(self, id, stops, delay=0):
        """Add a train to the network."""
        print(f"Adding train {id} to the network delay: {delay}")
        train = Train(id, stops, delay)
        self.trains.append(train)
    

    def simulate_delays(self, delta_t, steps):
        """Simulate delay propagation for a given number of time steps."""
        for train in self.trains:
            train.start_journey()
        for step in range(steps):
            print(f"Time Step {step + 1}:")
            
            # Update trains in transit
            for train in self.trains:
                train.update_travel()
            
            print("-" * 40)


In [35]:

# Example Usage
network = Network()

# Add stations
network.add_station('a')
network.add_station('b')
network.add_station('c')
network.add_station('d')
network.add_station('e')
network.add_station('f')

#trains
# train 1 stops
stops1 = [(network.stations['a'], 0, 0), (network.stations['b'], 5, 2), (network.stations['c'], 10, 0)]
network.add_train(1, stops1, delay=2)

# train 2 stops
stops2 = [(network.stations['a'], 0, 0), (network.stations['b'], 7, 3), (network.stations['e'], 15, 0)]
network.add_train(2, stops2, delay=3)

# train 3 stops
stops3 = [(network.stations['a'], 0, 0), (network.stations['f'], 12, 0)]
network.add_train(3, stops3, delay=4)

# Simulate delay propagation
network.simulate_delays(delta_t=1, steps=50)


Adding train 1 to the network delay: 2
Train 1 created in train with delay 2.
Adding train 2 to the network delay: 3
Train 2 created in train with delay 3.
Adding train 3 to the network delay: 4
Train 3 created in train with delay 4.
Incoming train 1 with delay 2 added to station b., Station delay updated to 2.
Train 1 has started its journey from a to b.
Incoming train 2 with delay 3 added to station b., Station delay updated to 5.
Train 2 has started its journey from a to b.
Incoming train 3 with delay 4 added to station f., Station delay updated to 4.
Train 3 has started its journey from a to f.
Time Step 1:
----------------------------------------
Time Step 2:
----------------------------------------
Time Step 3:
----------------------------------------
Time Step 4:
----------------------------------------
Time Step 5:
Train 1 arrived at b with delay 2. Station delay updated to 3.
Train 1 has arrived at b.
----------------------------------------
Time Step 6:
----------------------